In [1]:
import plotly as plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [2]:
import json
import requests
from requests.auth import HTTPBasicAuth

username = 'nischalm' # Replace with YOUR USERNAME
api_key = 'CsT1npukHvDgDTDHyILX' # Replace with YOUR API KEY

auth = HTTPBasicAuth(username, api_key)
headers = {'Plotly-Client-Platform': 'python'}

plotly.tools.set_credentials_file(username=username, api_key=api_key)

def get_pages(username, page_size):
    url = 'https://api.plot.ly/v2/folders/all?user='+username+'&page_size='+str(page_size)
    response = requests.get(url, auth=auth, headers=headers)
    if response.status_code != 200:
        return
    page = json.loads(response.content)
    yield page
    while True:
        resource = page['children']['next'] 
        if not resource:
            break
        response = requests.get(resource, auth=auth, headers=headers)
        if response.status_code != 200:
            break
        page = json.loads(response.content)
        yield page
        
def permanently_delete_files(username, page_size=500, filetype_to_delete='plot'):
    for page in get_pages(username, page_size):
        for x in range(0, len(page['children']['results'])):
            fid = page['children']['results'][x]['fid']
            res = requests.get('https://api.plot.ly/v2/files/' + fid, auth=auth, headers=headers)
            res.raise_for_status()
            if res.status_code == 200:
                json_res = json.loads(res.content)
                if json_res['filetype'] == filetype_to_delete:
                    # move to trash
                    requests.post('https://api.plot.ly/v2/files/'+fid+'/trash', auth=auth, headers=headers)
                    # permanently delete
                    requests.delete('https://api.plot.ly/v2/files/'+fid+'/permanent_delete', auth=auth, headers=headers)

#permanently_delete_files('nischalm', filetype_to_delete='plot')
#permanently_delete_files('nischalm', filetype_to_delete='grid')

In [3]:
import pandas as pd

In [4]:
prices = pd.read_csv('./prices.csv')

In [5]:
prices.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

|.|column|type|
|-|-|-|
|1|date|key|
|2|symbol|key|
|3|open|value|
|4|close|value|
|5|low|value|
|6|high|value|
|7|volume|value|

`date` and `symbol` together make up the key for this table. We have duplicate dates and symbols, but unique combination of date and symbol. The others are all value attritubes. As one of the key is time, this dataset has time-varing semantics.

## TASK
* task: present stock movements of tech companies over a prediod of time to client
* action: present, browse, compare
* target: trend (running average of stock over 30 days)

Say a stock broker wants to convince clients about buying the stock of a company. As a layman, the client would not understand the various terminologies, thus presenting a chart of running average (derived variable) of companies (two in this case) would help the client see why the stock would be good to buy. Here, we want to **present** to a client, the **trend** of stock (running average). This would be a **browse** search as we know the targets, but do not know the location. This would fall under **compare** as we try to see which stock is worth buying. 

### Viz 1

In [6]:
tech = prices[prices['symbol'].isin(['GOOG', 'AAPL'])]

In [7]:
tech_rolling = tech.groupby(tech.symbol)['open'].rolling(window=30).mean()

In [8]:
traces = []
for sym in ['GOOG', 'AAPL']:
    n = 'Google' if sym == 'GOOG' else 'Apple'
    sub = tech[tech['symbol'] == sym]
    trace1 = go.Scatter(
        x = sub['date'],
        y = tech_rolling[sym],
        name = n + '(Running average)',
        line = dict(
            color = 'blue' if sym == 'GOOG' else 'black'
        )
    )
    traces.append(trace1)
    trace2 = go.Scatter(
        x = sub['date'],
        y = sub['open'],
        opacity=0.5,
        name = n + '(true)',
        line = dict(
            color = 'blue' if sym == 'GOOG' else 'black',
            dash = 'dash'
        )
    )
    traces.append(trace2)
    
layout = dict(
    title = 'Stock prices (open) for Google and Apple',
    xaxis = dict(title='Date'),
    yaxis = dict(title='Price ($)')
)

fig = dict(data=traces, layout=layout)

In [9]:
py.iplot(fig)

/home/nischal/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In this viz, the marks are lines and channels are position(both), color, and type of line. I use the position channel as it is the most effective when showing quantitative values and to compare. I use color to distinguish the companies, and type of line to show the type of value plotted.

This plot shows the trend of the two companies (Google and Apple) over the years. It is easy to see that Apple prices have a lower movement as comapred to Google. Thus, we would but Google stocks for high returns (short period), and Apple stocks for a stable long term investment.

Here, the interactivity needed would be zooming and filter, which are both supported bu plotly. Zooming can be used to show historical time points where the stock follows the same trend, and filter to help the client see as much information as needed.

### Viz 2

In [18]:
tech = prices[prices['symbol'].isin(['MSFT', 'FB', 'GOOG', 'AAPL'])]

In [22]:
traces = []
def namer(sym):
    if sym == 'MSFT':
        return "Microsoft"
    if sym == 'FB':
        return 'Facebook'
    if sym == 'GOOG':
        return 'Google'
    if sym == 'AAPL':
        return "Apple"
    
for sym in ['MSFT', 'FB', 'GOOG', 'AAPL']:
    n = namer(sym)
    sub = tech[tech['symbol'] == sym]
    trace1 = go.Scatter(
        x = sub['date'],
        y = sub['open'],
        name = n,
        opacity=0.6
    )
    traces.append(trace1)
    
layout = dict(
    title = 'Stock prices (open) for Microsoft and Facebook',
    xaxis = dict(title='Date'),
    yaxis = dict(title='Price ($)')
)

fig = dict(data=traces, layout=layout)

In [23]:
py.iplot(fig)

Here, I only use lines as marks and position(both) and color as channels. As above, position is best used for comparing. 

It is easy to see that Facebook stocks are rather new, but have a very high movement. Whereas Miscrosoft stocks have a lower movement, but are much more stable. Similiar to the above example, FB would be good for high return in a short period, but MSFT would be better for a long term inverstment.

Similar to the previous viz, zooming and filter are available.